In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#!pip install rouge_score
#!pip install transformers
#!pip install  datasets
#https://stackoverflow.com/questions/73781510/how-to-get-the-logits-for-the-t5-model-when-using-the-generate-method-for-infe

In [1]:
numSelected = 20
batchSize = 40
max_length_in = 1024
max_length_out = 128
path="C:/Users/Za/Desktop/SchoolWork/Nam 4/Seminar/NLU_TextSummarization/Train_Code/"
checkpoint = path+"1024-128-TrainSplit/"

In [2]:
from rouge_score import rouge_scorer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset
import numpy as np
import time

In [95]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print ("Model is running on", device)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, local_files_only=True, output_hidden_states=True).to(device)
tokenizer = AutoTokenizer.from_pretrained(checkpoint, local_files_only=True, output_hidden_states=True)

Model is running on cuda


In [96]:
#Predict mot input duy nhat
input_text = "You can't just replace the processor on the UNO with something better. The 328P is pretty much the best MCU there is in the pinout for that board.Instead you can replace the whole UNO with something more powerful. My board of choice for AI work would have to be one based on the Kendryte K210 chip, such as the Maixduino (which is helpfully in an UNO footprint). The K210 is a dual-core 64-bit 400MHz RISC-V CPU with embedded neural network co-processor. On the Maixduino it's also coupled with an ESP32 for WiFi/Bluetooth. That's 2x 400MHz 64-bit cores, 2x 32-bit 240MHz cores, one low-power FSM core, and a neural network core all on one board."
input_text = input_text*2
def predictOneSentence (input_text, model, tokenizer, max_length_in = 512, max_length_out = 128):
    inputs = tokenizer.encode("summarize: " + input_text, return_tensors="pt", max_length=max_length_in, truncation=True).to(device)
    with torch.no_grad():
        summary_ids = model.generate(inputs, max_length=max_length_out, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True, output_hidden_states=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        fuckme = model.forward (input_ids=inputs, labels=inputs, output_hidden_states =True)
        return summary, fuckme, summary_ids

    
summary, modelOutput, summary_ids = predictOneSentence (input_text, model, tokenizer, max_length_in, max_length_out)
#summary = testPredict (input_text, model, tokenizer, max_length_in, max_length_out)
print("Summary:", summary)

Summary: 328P is pretty much the best MCU there is in the pinout for that board. The K210 is a dual-core 64-bit 400MHz RISC-V CPU with embedded neural network co-processor. The K210 is a dual-core 64-bit 400MHz RISC-V CPU with embedded neural network co-processor.


In [97]:
print (type (modelOutput))
print ("summary_ids: ",summary_ids)
#print (summary_ids['hidden_states'])
print("Key:", modelOutput.keys())
#print (model)
#print("content:", modelOutput['decoder_hidden_states'])
#print("tokenizer.decode:", tokenizer.decode(modelOutput[0], skip_special_tokens=True))

<class 'transformers.modeling_outputs.Seq2SeqLMOutput'>
summary_ids:  tensor([[    0,     3, 28070,   345,    19,  1134,   231,     8,   200,   283,
          5211,   132,    19,    16,     8,  4522,   670,    21,    24,  1476,
             5,    37,   480, 15239,    19,     3,     9,  7013,    18,  9022,
          6687,    18,  2360,  4837, 20210,     3, 13431,   254,    18,   553,
         13823,    28, 13612, 24228,  1229,   576,    18, 15056,   127,     5,
            37,   480, 15239,    19,     3,     9,  7013,    18,  9022,  6687,
            18,  2360,  4837, 20210,     3, 13431,   254,    18,   553, 13823,
            28, 13612, 24228,  1229,   576,    18, 15056,   127,     5,     1]],
       device='cuda:0')
Key: odict_keys(['loss', 'logits', 'past_key_values', 'decoder_hidden_states', 'encoder_last_hidden_state', 'encoder_hidden_states'])


In [98]:
with open('model.txt', 'w') as f:
    print(model, file=f)  # Python 3.x

In [99]:
print ("Loss: ", modelOutput['loss'])

Loss:  tensor(4.5415, device='cuda:0')


In [100]:
print("logits shape:", modelOutput['logits'].shape)
print("logits size:", len(modelOutput['logits'][0]))
print("logits size:", len(modelOutput['logits'][0][0]))

logits shape: torch.Size([1, 383, 32128])
logits size: 383
logits size: 32128


In [101]:
print("past_key_values shape is: (6, 4, 1, 8, 193, 64)")
print("past_key_values size:", len(modelOutput['past_key_values']))
print("past_key_values[0] size:", len(modelOutput['past_key_values'][0]))
print("past_key_values[0] size:", len(modelOutput['past_key_values'][0][0]))
print("past_key_values[0] size:", len(modelOutput['past_key_values'][0][0][0]))
print("past_key_values[0] size:", len(modelOutput['past_key_values'][0][0][0][0]))
print("past_key_values[0] size:", len(modelOutput['past_key_values'][0][0][0][0][0]))

past_key_values shape is: (6, 4, 1, 8, 193, 64)
past_key_values size: 6
past_key_values[0] size: 4
past_key_values[0] size: 1
past_key_values[0] size: 8
past_key_values[0] size: 383
past_key_values[0] size: 64


In [102]:
print("decoder_hidden_states shape is: (7, 1, 193, 512)")
print("encoder_last_hidden_state size:", modelOutput['encoder_last_hidden_state'].shape)
print("encoder_hidden_states shape is: (7, 1, 193, 512)")
#print("decoder_hidden_states size:", len(modelOutput['decoder_hidden_states']))
#print("decoder_hidden_states size:", modelOutput['decoder_hidden_states'][0].shape)
#print("encoder_hidden_states size:", len(modelOutput['encoder_hidden_states']))
#print("encoder_hidden_states size:", modelOutput['encoder_hidden_states'][0].shape)

decoder_hidden_states shape is: (7, 1, 193, 512)
encoder_last_hidden_state size: torch.Size([1, 383, 512])
encoder_hidden_states shape is: (7, 1, 193, 512)


In [103]:

print("decoder_hidden_states 0 layer size:", len(modelOutput['decoder_hidden_states'][0][0]))
print("decoder_hidden_states 1 layer size:", len(modelOutput['decoder_hidden_states'][1][0]))
print("decoder_hidden_states 2 layer size:", len(modelOutput['decoder_hidden_states'][2][0]))
print("decoder_hidden_states 3 layer size:", len(modelOutput['decoder_hidden_states'][3][0]))
print("decoder_hidden_states 4 layer size:", len(modelOutput['decoder_hidden_states'][4][0]))
print("decoder_hidden_states 5 layer size:", len(modelOutput['decoder_hidden_states'][5][0]))
print("decoder_hidden_states 6 layer size:", len(modelOutput['decoder_hidden_states'][6][0]))

decoder_hidden_states 0 layer size: 383
decoder_hidden_states 1 layer size: 383
decoder_hidden_states 2 layer size: 383
decoder_hidden_states 3 layer size: 383
decoder_hidden_states 4 layer size: 383
decoder_hidden_states 5 layer size: 383
decoder_hidden_states 6 layer size: 383


In [104]:
encoder_last_hidden_state_np = modelOutput['encoder_last_hidden_state'].cpu().numpy()
encoder_last_hidden_state_np.shape

(1, 383, 512)

In [73]:
raise Exception ("FUCKME")

Exception: FUCKME

In [7]:
def getRougeScore (dataset, model, tokenizer, max_length_in = 512, max_length_out = 128, batchSize = 75):
    start_time_2 = time.time()
    print ("Number of sample: ", len(dataset),", Batch size: ", batchSize)
    multiple_text = dataset["text"]
    mulitple_summary = dataset ["summary"]
    outputList = []
    start_time = time.time()
    
    for x in range (0, numSelected, batchSize):
        tokenized_text = tokenizer(multiple_text[x:min(numSelected, x+batchSize)], return_tensors="pt", 
                                   max_length=max_length_in, truncation=True, padding = True).to(device)
        nigger = model.generate(**tokenized_text, max_length=max_length_out, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
        outputList.extend (nigger)
        
    print ("model.generate Total time taken: ",  time.time() - start_time,", Average time for 1 sample: ", (time.time() - start_time) / numSelected)
    summary = [tokenizer.decode(x, skip_special_tokens=True) for x in outputList]
    
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1_scores, rouge2_scores, rougeL_scores = [], [], [] #Define multiple empty list
    for generated_summary, reference_summary in zip (summary, mulitple_summary):
        scores = scorer.score(generated_summary, reference_summary)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)
    avg_rouge1 = np.mean(rouge1_scores)
    avg_rouge2 = np.mean(rouge2_scores)
    avg_rougeL = np.mean(rougeL_scores)
    
    print(f"Average ROUGE-1: {avg_rouge1:.4f}")
    print(f"Average ROUGE-2: {avg_rouge2:.4f}")
    print(f"Average ROUGE-L: {avg_rougeL:.4f}")
    print ("Total time taken: ",  time.time() - start_time_2)
    return avg_rouge1, avg_rouge2, avg_rougeL

In [8]:
try:
    #legalDataset
    billsumDataset
    #cnnDataset
except NameError:
    print("Dataset not defined, loading it...")
    #legalDataset = load_dataset("ccdv/govreport-summarization", split="test")
    billsumDataset = load_dataset("billsum", split="test")
    #cnnDataset = load_dataset("cnn_dailymail", '3.0.0', split= "test")
else:
    print("Dataset already defined, skipping it...")

Dataset not defined, loading it...


In [9]:
#first_100_billsum = billsumDataset.select(range(numSelected))
#getRougeScore (first_100_billsum, model, tokenizer, max_length_in, max_length_out, batchSize)
numSelected = len (billsumDataset)
getRougeScore (billsumDataset, model, tokenizer, max_length_in, max_length_out, batchSize)

Number of sample:  3269 , Batch size:  40
model.generate Total time taken:  1410.955011844635 , Average time for 1 sample:  0.431616705978781
Average ROUGE-1: 0.4878
Average ROUGE-2: 0.3168
Average ROUGE-L: 0.3827
Total time taken:  1437.8629744052887


(0.4878147369736801, 0.3167770930817783, 0.3827119994133303)

In [34]:
print ("Input text:")
#102, 5
selected_idx = 7
tokenizer_target = billsumDataset[selected_idx]["text"]
input_text = tokenizer.decode(tokenizer(text_target=tokenizer_target, max_length=max_length_out, truncation=True)['input_ids'], skip_special_tokens=True)
print (input_text)
summary = predictOneSentence (billsumDataset[selected_idx]["text"], model, tokenizer, max_length_in, max_length_out)
print("Predicted summary:\n", summary)
#true_summary_token = 
tokenizer_target = billsumDataset[selected_idx]["summary"]
true_summary = tokenizer.decode(tokenizer(text_target=tokenizer_target, max_length=max_length_out, truncation=True)['input_ids'], skip_special_tokens=True)
print("True summary:\n", true_summary)

Input text:
SECTION 1. SHORT TITLE. This Act may be cited as the Special Agent Scott K. Carey Public Safety Officer Benefits Enhancement Act''. TITLE I--EDUCATIONAL ASSISTANCE TO OFFICERS DISABLED IN THE LINE OF DUTY SEC. 101. BASIC ELIGIBILITY. Section 1212(a)(1) of the Omnibus Crime Control and Safe Streets Act of 1968 (42 U.S.C. 3796d-1(a)(1)) is amended--
Predicted summary:
 Special Agent Scott K. Carey Public Safety Officer Benefits Enhancement Act - Amends the Omnibus Crime Control and Safe Streets Act of 1968 to revise the definition of "eligible dependent" to include any public safety officer who dies in the line of duty on or after January 1, 1978, or becomes permanently and totally disabled as the direct result of a catastrophic injury sustained in the line of duty on or after January 1, 1978. Requires the Bureau of Investigation (BIA) to pay an annual pension to one or more survivors of the deceased public safety officer. Requires
True summary:
 Special Agent Scott K. Carey 

In [ ]:
raise ValueError('Stop running the code below')
legalDataset = legalDataset.rename_column("report", "text")
first_100_gov = legalDataset.select(range(numSelected))
len(first_100_gov)
getRougeScore (first_100_gov, model, tokenizer, max_length_in, max_length_out, batchSize)

In [ ]:
cnnDataset = cnnDataset.rename_column("article", "text")
cnnDataset = cnnDataset.rename_column("highlights", "summary")
first_100_cnn = cnnDataset.select(range(numSelected))
getRougeScore (first_100_cnn, model, tokenizer, max_length_in, max_length_out, batchSize)

In [ ]:
raise ValueError('Stop running the code below')

In [ ]:
#DO NOT RUN THIS CODE
#ONLY MADE FOR TIME COMPARISON
import time
numSelected = 30
first_100_billsum = billsum.select(range(numSelected))
print ("Number of sample: ", len(first_100_billsum))

start_time_2 = time.time()

multiple_text = first_100_billsum["text"]
mulitple_summary = first_100_billsum ["summary"]
tokenized_text = tokenizer(multiple_text, return_tensors="pt", max_length=512, truncation=True).to(device)
start_time = time.time()
ayy = model.generate(**tokenized_text, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
averageTime = (time.time() - start_time) / numSelected
print ("model.generate Total time taken: ",  time.time() - start_time,", Average time for 1 sample: ", averageTime)
summary = [tokenizer.decode(x, skip_special_tokens=True) for x in ayy]
ayy = zip (summary, mulitple_summary)
for generated_summary, reference_summary in ayy:
    scores = scorer.score(generated_summary, reference_summary)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)
avg_rouge1 = np.mean(rouge1_scores)
avg_rouge2 = np.mean(rouge2_scores)
avg_rougeL = np.mean(rougeL_scores)
print(f"Average ROUGE-1: {avg_rouge1:.4f}")
print(f"Average ROUGE-2: {avg_rouge2:.4f}")
print(f"Average ROUGE-L: {avg_rougeL:.4f}")

print ("Optimized Total time taken: ",  time.time() - start_time_2)

start_time_2 = time.time()

for example in first_100_billsum:
    text = example["text"]
    reference_summary = example["summary"]
    # Generate summary
    generated_summary = predictOneSentence(text, model, tokenizer)
    # Calculate ROUGE scores
    scores = scorer.score(generated_summary, reference_summary)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)
avg_rouge1 = np.mean(rouge1_scores)
avg_rouge2 = np.mean(rouge2_scores)
avg_rougeL = np.mean(rougeL_scores)
print(f"Average ROUGE-1: {avg_rouge1:.4f}")
print(f"Average ROUGE-2: {avg_rouge2:.4f}")
print(f"Average ROUGE-L: {avg_rougeL:.4f}")

print ("Unoptimized Total time taken: ",  time.time() - start_time_2)